In [68]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.metrics.pairwise import rbf_kernel, sigmoid_kernel, polynomial_kernel
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from tempfile import TemporaryFile
import pandas as pd
import matplotlib.pyplot as plt
from itertools import product
import numpy as np
import sklearn
import time
import random
import json
import os
from sklearn.preprocessing import KernelCenterer
# import sys
# maindir = '/'.join(os.getcwd().split('/')[:-1])
# sys.path.append(maindir)
from source.utils import make_symmetric_matrix_psd, normalize

## Calculate kernel matrices per parameter and kernel combination  

In [2]:
maindir = '/'.join(os.getcwd().split('/')[:-1])
kerneldir = os.path.join(maindir, 'data/precomputed_multitask_kernels')
input_file = os.path.join(maindir, "data/kernel_parameter.csv")
kernel_param = np.array(pd.read_csv(input_file))

In [3]:
kernel_param

array([['poly_degree', 2.0, 6.0],
       ['alpha_sigmoid', 0.25, 0.75],
       ['offset_sigmoid', -2.0, 3.0],
       ['sigma_rbf', -6.0, 6.0],
       ['C', -6.0, 5.0]], dtype=object)

In [120]:
def make_kernel_combinations(
    meta_data: np.ndarray,
    kernel_dosage: str,
    kernel_abSignal: str
) -> dict:
    """Make dictionary of kernel combination values

    For each kernel parameter a range of values is defined.

    Parameters
    ----------
    meta_data : np.ndarray 
        Matrix of kernel parameter ranges.
    kernel_dosage : str
        Pre-defined kernel function for to represent relationship between individuals based on dosage level.
    kernel_abSignal : str
        Pre-defined kernel function for to represent relationship between individuals based on ab signal intensity

    Returns
    -------
    kernel_comb_param : dict
        Dictionary of value ranges for each kernel parameter.

    """
    if kernel_dosage == "rbf_kernel" and kernel_abSignal == "rbf_kernel":
        kernel_comb_param = {"SA": np.arange(meta_data[1, 1], 1, meta_data[1, 1]),
                             "SO": np.arange(meta_data[2, 1], meta_data[2, 2]),
                             "R1": 10 ** np.arange(meta_data[3, 1], meta_data[3, 2], dtype=float),
                             "R2": 10 ** np.arange(meta_data[3, 1], meta_data[3, 2], dtype=float),
                             "P1": np.array(['X']),
                             "P2": np.array(['X'])}
        
    elif kernel_dosage == "poly_kernel" and kernel_abSignal == "poly_kernel":
        kernel_comb_param = {"SA": np.arange(meta_data[1, 1], 1, meta_data[1, 1]),
                             "SO": np.arange(meta_data[2, 1], meta_data[2, 2], dtype=float),
                             "R1": np.array(['X']),
                             "R2": np.array(['X']),
                             "P1": np.arange(meta_data[0, 1], meta_data[0, 2]),
                             "P2": np.arange(meta_data[0, 1], meta_data[0, 2])}  
        
    elif kernel_dosage == "poly_kernel" and kernel_abSignal == "rbf_kernel":
        kernel_comb_param = {"SA": np.arange(meta_data[1, 1], 1, meta_data[1, 1]),
                             "SO": np.arange(meta_data[2, 1], meta_data[2, 2]),
                             "R1": np.array(['X']),
                             "R2": 10 ** np.arange(meta_data[3, 1], meta_data[3, 2], dtype=float),
                             "P1": np.arange(meta_data[0, 1], meta_data[0, 2]),
                             "P2": np.array(['X'])}
    else:  kernel_comb_param = {"SA": np.arange(meta_data[1, 1], 1, meta_data[1, 1]),
                             "SO": np.arange(meta_data[2, 1], meta_data[2, 2]),
                             "R1": 10 ** np.arange(meta_data[3, 1], meta_data[3, 2], dtype=float),
                             "R2": np.array(['X']),
                             "P1": np.array(['X']),
                             "P2": np.arange(meta_data[0, 1], meta_data[0, 2])}
    return kernel_comb_param

In [6]:
def multitask(
    a: np.ndarray,
    b: np.ndarray,
) -> np.ndarray:
    """ Multitask approach

    Combination of two kernel matrices are combined by element-wise multiplication to one kernel matrix.

    Parameters
    ----------
    a : np.ndarray
        Kernel matrix a.
    b : np.ndarray
        Kernel matrix b.

    Returns
    -------
    a * b : np.ndarray
        Element-wise multiplicated kernel matrix a * b.
    """
    return a * b

In [169]:
def make_kernel_matrix(
    data: pd.DataFrame,
    model: dict,
    kernel_dosage: str,
    kernel_abSignals: str,
    scale: bool = False,
) -> np.ndarray:
    """ Compute combined kernel matrix

    for each task, time, PfSPZ dose and antibody signal intensity a kernel matrix is pre-computed given the pre-defined
    kernel function and combined to one matrix by the multitask approach

    Parameters
    ----------
    data : pd.Dataframe
        Pre-processed proteome data.
    model : dict
        Combination of kernel parameters.
    kernel_dosage : str
        Kernel function to compute relationship between individuals based on dosage level.
    kernel_abSignals : str
        Kernel function to compute relationship between individuals based on antibody signal intensity.
    scale : bool, default=False
        If True, scale the combined matrix by first centering it followed by normalization.

    Returns
    -------
    multi_AB_signals_time_dose_kernel_matrix : np.ndarray
        Returns a combined matrix.
    """

    # get start point of antibody reactivity signals in data (n_p)
    AB_signal_start = data.columns.get_loc("TimePointOrder") + 1
    AB_signals = data[data.columns[AB_signal_start:]]

    # extract time points to vector (y_t)
    time_series = data["TimePointOrder"]

    # extract dosage to vector (y_d)
    dose = data["Dose"]

    if kernel_dosage == "rbf_kernel" and kernel_abSignals == "rbf_kernel":
        # pre-computed kernel matrix of time points K(n_t,n_t')
        time_series_kernel_matrix = make_symmetric_matrix_psd(
            sigmoid_kernel(
                time_series.values.reshape(len(time_series), 1),
                gamma=model[0],
                coef0=model[1],
            )
        )
        # pre-computed kernel matrix of antibody reactivity K(n_p,n_p')
        AB_signals_kernel_matrix = make_symmetric_matrix_psd(rbf_kernel(AB_signals, gamma=model[3]))
        # pre-computed kernel matrix of dosage K(n_d,n_d')
        dose_kernel_matrix = make_symmetric_matrix_psd(
            rbf_kernel(dose.values.reshape(len(dose), 1), gamma=model[2])
        )
    else:
        time_series_kernel_matrix = make_symmetric_matrix_psd(
            sigmoid_kernel(
                time_series.values.reshape(len(time_series), 1),
                gamma=model[0],
                coef0=model[1],
            )
        )
        # show_heatmap(time_series_kernel_matrix)

        if kernel_abSignals == "rbf_kernel":
            AB_signals_kernel_matrix = make_symmetric_matrix_psd(rbf_kernel(AB_signals, gamma=model[3]))

        elif kernel_abSignals == "poly_kernel":
            AB_signals_kernel_matrix = make_symmetric_matrix_psd(
                polynomial_kernel(AB_signals, degree=model[5])
            )
        # show_heatmap(AB_signals_kernel_matrix)

        # pre-computed kernel matrix of dosage K(n_d,n_d')
        if kernel_dosage == "rbf_kernel":
            dose_kernel_matrix = make_symmetric_matrix_psd(
                rbf_kernel(dose.values.reshape(len(dose), 1), gamma=model[2])
            )
        elif kernel_dosage == "poly_kernel":
            dose_kernel_matrix = make_symmetric_matrix_psd(
                polynomial_kernel(dose.values.reshape(len(dose), 1), degree=model[4])
            )
        # show_heatmap(dose_kernel_matrix)

    # pre-compute multitask kernel matrix K((np, nt),(np', nt'))
    multi_AB_signals_time_series_kernel_matrix = make_symmetric_matrix_psd(
        multitask(
            AB_signals_kernel_matrix,
            time_series_kernel_matrix,
        )
    )
    # show_heatmap(multi_AB_signals_time_series_kernel_matrix)

    # pre-compute multitask kernel matrix K((np, nt, nd),(np', nt', nd'))
    multi_AB_signals_time_dose_kernel_matrix = make_symmetric_matrix_psd(
        multitask(
            multi_AB_signals_time_series_kernel_matrix,
            dose_kernel_matrix,
        )
    )
    if scale:
        multi_AB_signals_time_dose_kernel_matrix = normalize(
            KernelCenterer().fit_transform(multi_AB_signals_time_dose_kernel_matrix)
        )
    # show_heatmap(multi_AB_signals_time_dose_kernel_matrix)

    # proof Dimension and rank of kernel matrix
    print("Dimension of kernel matrix")
    print(multi_AB_signals_time_dose_kernel_matrix.shape)
    print("Rank of kernel matrix")
    print(np.linalg.matrix_rank(multi_AB_signals_time_dose_kernel_matrix))
    

    return multi_AB_signals_time_dose_kernel_matrix


## Make kernel parameter combinations 

In [121]:
kernel_comb_rbf_rbf = make_kernel_combinations(kernel_param, "rbf_kernel", "rbf_kernel")

In [122]:
kernel_comb_rbf_rbf

{'SA': array([0.25, 0.5 , 0.75]),
 'SO': array([-2., -1.,  0.,  1.,  2.]),
 'R1': array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01,
        1.e+02, 1.e+03, 1.e+04, 1.e+05]),
 'R2': array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01,
        1.e+02, 1.e+03, 1.e+04, 1.e+05]),
 'P1': array(['X'], dtype='<U1'),
 'P2': array(['X'], dtype='<U1')}

In [123]:
kernel_comb_poly_poly = make_kernel_combinations(kernel_param, "poly_kernel", "poly_kernel")

In [124]:
kernel_comb_poly_poly

{'SA': array([0.25, 0.5 , 0.75]),
 'SO': array([-2., -1.,  0.,  1.,  2.]),
 'R1': array(['X'], dtype='<U1'),
 'R2': array(['X'], dtype='<U1'),
 'P1': array([2., 3., 4., 5.]),
 'P2': array([2., 3., 4., 5.])}

In [125]:
kernel_comb_rbf_poly = make_kernel_combinations(kernel_param, "rbf_kernel", "poly_kernel")

In [126]:
kernel_comb_rbf_poly

{'SA': array([0.25, 0.5 , 0.75]),
 'SO': array([-2., -1.,  0.,  1.,  2.]),
 'R1': array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01,
        1.e+02, 1.e+03, 1.e+04, 1.e+05]),
 'R2': array(['X'], dtype='<U1'),
 'P1': array(['X'], dtype='<U1'),
 'P2': array([2., 3., 4., 5.])}

In [127]:
kernel_comb_poly_rbf = make_kernel_combinations(kernel_param, "poly_kernel", "rbf_kernel")

In [128]:
kernel_comb_poly_rbf

{'SA': array([0.25, 0.5 , 0.75]),
 'SO': array([-2., -1.,  0.,  1.,  2.]),
 'R1': array(['X'], dtype='<U1'),
 'R2': array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01,
        1.e+02, 1.e+03, 1.e+04, 1.e+05]),
 'P1': array([2., 3., 4., 5.]),
 'P2': array(['X'], dtype='<U1')}

### Save kernel parameter combinations as grid

In [149]:
with open('kernel_comb_SRR.json', 'w') as fp:
    json.dump({k: v.tolist() for k, v in kernel_comb_rbf_rbf.items()}, fp)

In [150]:
with open('kernel_comb_SPP.json', 'w') as fp:
    json.dump({k: v.tolist() for k, v in kernel_comb_poly_poly.items()}, fp)

In [151]:
with open('kernel_comb_SRP.json', 'w') as fp:
    json.dump({k: v.tolist() for k, v in kernel_comb_rbf_poly.items()}, fp)

In [152]:
with open('kernel_comb_SPR.json', 'w') as fp:
    json.dump({k: v.tolist() for k, v in kernel_comb_poly_rbf.items()}, fp)

## Make kernel matrices with different kernel combinations

## Whole proteome data

In [35]:
input_data = os.path.join(maindir, "data/preprocessed_whole_data.csv")
data_malaria = pd.read_csv(input_data)

In [36]:
data_malaria.head()

,Patient,group,Protection,Dose,TimePointOrder,mal_mito_3.iso1.exon1.amp1_4066,mal_mito_2.iso1.exon1.amp1_1466,mal_mito_1.iso1.exon1.amp1_3777,PFL2430c_484,PFL2215w_4059,...,MAL8P1.320_1o1_4243,MAL8P1.156_187,MAL8P1.155_2o3_4890,MAL8P1.143_3o15_2092,MAL8P1.143_1o15_2416,MAL8P1.143_14o15_1450,MAL8P1.140_4049,MAL7P1.32_5037,MAL13P1.303e4s1_1829,MAL13P1.303e1s1_4421
0,T2-002 C -1,2,1,1,3,6.727920,0.000000,6.409391,6.768184,0.000000,...,0.000000,0.000000,0.000000,10.651052,8.918863,0.000000,6.569856,14.609121,7.679480,2.807355
1,T2-002 C 28,2,1,1,4,6.348728,6.954196,6.238405,10.194141,10.168045,...,9.016808,9.658211,9.852530,11.159556,10.738937,10.656425,8.092757,15.251630,10.368506,9.494856
2,T2-002 III 14,2,1,1,2,7.033423,6.357552,6.954196,3.700440,0.000000,...,0.000000,0.000000,0.000000,11.168672,9.628446,0.000000,0.000000,15.327447,0.000000,0.000000
3,T2-005 C -1,5,0,0,3,9.262095,9.241983,9.487840,6.539159,9.579316,...,8.308339,9.697836,9.267957,8.409391,9.208234,8.105909,8.643856,9.227616,9.048487,7.055282
4,T2-005 C 28,5,0,0,4,7.734710,7.891784,7.954196,13.478011,8.794416,...,7.471675,8.873444,9.080818,9.632995,6.339850,8.233620,10.639793,15.874609,8.668885,8.531381


## Selective proteome data set

In [55]:
#load file via pandas as csv file 
input_file_s = "/Users/schmidtj/Documents/BenjaminsDaten/firstAnalysis/Proteinarray/T2A/Multitask_time_dosis_signals/Proteome_chip_analysis/Paramter_validation_for_each_kernel_matrix/preprocessed_MalariaProteom_chip_data/preprocessded_selected_data_malaria_set_minusBS.csv"
selected_data_malaria = pd.read_csv(input_file_s)



In [56]:
selected_data_malaria.head()

,Patient,group,Protection,Dose,TimePointOrder,PFE1590w_2119,PFD0425w_2780,PFC0210c_2763,PFB0300c_4064,PF3D7_1480000.iso1.exon2.amp1_2445,...,PF3D7_0100100.iso1.exon1.amp2_1950,PF3D7_0100100.iso1.exon1.amp1_1661,PF14_0540_1132,PF13_0201_3400,PF13_0197_176,PF13_0172_4696,PF13_0012_499,PF11_0344_1786,PF10_0088_4048,PF08_0013_1803
0,T2-002 C -1,2,1,1,3,0.000000,9.567005,9.556506,12.760824,7.672425,...,0.000000,0.000000,14.453142,10.540612,10.616549,7.055282,5.954196,0.000000,8.299208,0.000000
1,T2-002 C 28,2,1,1,4,9.159871,9.714246,10.738092,12.777974,8.727920,...,6.491853,6.098032,14.970421,11.440869,13.282292,10.150382,7.751544,12.554109,9.832890,9.711667
2,T2-002 III 14,2,1,1,2,0.000000,0.000000,8.763212,12.830218,8.477758,...,0.000000,2.906891,15.156320,10.179909,11.091435,0.000000,0.000000,0.000000,0.000000,0.000000
3,T2-005 C -1,5,0,0,3,9.381543,0.000000,10.688687,8.436712,10.115044,...,9.682995,8.867279,0.000000,9.858758,10.399278,9.202124,8.353147,8.233620,9.217958,8.586840
4,T2-005 C 28,5,0,0,4,11.328114,8.471675,8.625709,11.577429,9.773963,...,8.044394,7.426265,14.782461,8.590587,8.252665,7.370687,7.968667,8.471675,9.381543,6.864186


In [23]:
def sort(
    data: pd.DataFrame,
) -> pd.DataFrame:
    """ Sorting

    Input data is sorted by time point to keep same patient over all four time points in order.

    Parameters
    ----------
    data : pd.DataFrame
        Raw proteome data, n x m pd.DataFrame (n = samples as rows, m = features as columns)

    Returns
    -------
    data : pd.DataFrame
        Returns sorted DataFrame
    """
    data.sort_values(by=["TimePointOrder","Patient"], inplace=True)
    data.reset_index(inplace = True)
    data.drop(columns = ['index'], inplace=True)
    return data

In [157]:
def main(
    data: pd.DataFrame,
    kernel_param: dict,
    kernel_for_dosage: str,
    kernel_for_abSignal: str,
    scale: bool = False,
):
    
    
    if kernel_for_dosage == "rbf_kernel" and kernel_for_abSignal == "rbf_kernel":
        for m in product(kernel_param["SA"], kernel_param["SO"], kernel_param["R1"], kernel_param["R2"], kernel_param["P1"], kernel_param["P2"]):
            kernel_matrix = make_kernel_matrix(
                data,
                model=m,
                kernel_dosage=kernel_for_dosage,
                kernel_abSignals=kernel_for_abSignal,
                scale=scale,
            )
            output_file_name = "kernel_matrix_SRR" + "_SA_" + str(m[0]) + "_SO_" + str(m[1]) +\
                               "_R1_" + str('{:.1E}'.format(m[2])) + "_R2_" + str('{:.1E}'.format(m[3])) +\
                               "_P1_" + str(m[4]) + "_P2_" + str(m[5]) + ".npy"
            np.save(output_file_name, kernel_matrix)
            # alternarive float name fuer kernel param str('{:.6f}'.format(m[2])) mit 6 nachkommastellen 
            
    elif kernel_for_dosage == "poly_kernel" and kernel_for_abSignal == "rbf_kernel":
        for m in product(kernel_param["SA"], kernel_param["SO"], kernel_param["R1"], kernel_param["R2"], kernel_param["P1"], kernel_param["P2"]):
            kernel_matrix = make_kernel_matrix(
                data,
                model=m,
                kernel_dosage=kernel_for_dosage,
                kernel_abSignals=kernel_for_abSignal,
                scale=scale,
            )
            output_file_name = "kernel_matrix_SPR" + "_SA_" + str(m[0]) + "_SO_" + str(m[1]) +\
                               "_R1_" + str(m[2]) + "_R2_" + str('{:.1E}'.format(m[3])) +\
                               "_P1_" + str(m[4]) + "_P2_" + str(m[5]) + ".npy"
            np.save(output_file_name, kernel_matrix)
            
    elif kernel_for_dosage == "rbf_kernel" and kernel_for_abSignal == "poly_kernel":
        for m in product(kernel_param["SA"], kernel_param["SO"], kernel_param["R1"], kernel_param["R2"], kernel_param["P1"], kernel_param["P2"]):
            kernel_matrix = make_kernel_matrix(
                data,
                model=m,
                kernel_dosage=kernel_for_dosage,
                kernel_abSignals=kernel_for_abSignal,
                scale=scale,
            )
            output_file_name = "kernel_matrix_SRP" + "_SA_" + str(m[0]) + "_SO_" + str(m[1]) +\
                               "_R1_" + str('{:.1E}'.format(m[2])) + "_R2_" + str(m[3]) +\
                               "_P1_" + str(m[4]) + "_P2_" + str(m[5]) + ".npy"
            np.save(output_file_name, kernel_matrix) 
            
    elif kernel_for_dosage == "poly_kernel" and kernel_for_abSignal == "poly_kernel":
        for m in product(kernel_param["SA"], kernel_param["SO"], kernel_param["R1"], kernel_param["R2"], kernel_param["P1"], kernel_param["P2"]):
            kernel_matrix = make_kernel_matrix(
                data,
                model=m,
                kernel_dosage=kernel_for_dosage,
                kernel_abSignals=kernel_for_abSignal,
                scale=scale,
            )
            output_file_name = "kernel_matrix_SPP" + "_SA_" + str(m[0]) + "_SO_" + str(m[1]) +\
                               "_R1_" + str(m[2]) + "_R2_" + str(m[3]) +\
                               "_P1_" + str(m[4]) + "_P2_" + str(m[5]) + ".npy"
            np.save(output_file_name, kernel_matrix) 
    

In [163]:
def main_selectiveSet(
    data: pd.DataFrame,
    kernel_param: dict,
    kernel_for_dosage: str,
    kernel_for_abSignal: str,
    scale: bool = False,
):
    
    
    if kernel_for_dosage == "rbf_kernel" and kernel_for_abSignal == "rbf_kernel":
        for m in product(kernel_param["SA"], kernel_param["SO"], kernel_param["R1"], kernel_param["R2"], kernel_param["P1"], kernel_param["P2"]):
            kernel_matrix = make_kernel_matrix(
                data,
                model=m,
                kernel_dosage=kernel_for_dosage,
                kernel_abSignals=kernel_for_abSignal,
                scale=scale,
            )
            output_file_name = "kernel_matrix_SelectiveSet_SRR" + "_SA_" + str(m[0]) + "_SO_" + str(m[1]) +\
                               "_R1_" + str('{:.1E}'.format(m[2])) + "_R2_" + str('{:.1E}'.format(m[3])) +\
                               "_P1_" + str(m[4]) + "_P2_" + str(m[5]) + ".npy"
            np.save(output_file_name, kernel_matrix)
            
            
    elif kernel_for_dosage == "poly_kernel" and kernel_for_abSignal == "rbf_kernel":
        for m in product(kernel_param["SA"], kernel_param["SO"], kernel_param["R1"], kernel_param["R2"], kernel_param["P1"], kernel_param["P2"]):
            kernel_matrix = make_kernel_matrix(
                data,
                model=m,
                kernel_dosage=kernel_for_dosage,
                kernel_abSignals=kernel_for_abSignal,
                scale=scale,
            )
            output_file_name = "kernel_matrix_SelectiveSet_SPR" + "_SA_" + str(m[0]) + "_SO_" + str(m[1]) +\
                               "_R1_" + str(m[2]) + "_R2_" + str('{:.1E}'.format(m[3])) +\
                               "_P1_" + str(m[4]) + "_P2_" + str(m[5]) + ".npy"
            np.save(output_file_name, kernel_matrix)
            
    elif kernel_for_dosage == "rbf_kernel" and kernel_for_abSignal == "poly_kernel":
        for m in product(kernel_param["SA"], kernel_param["SO"], kernel_param["R1"], kernel_param["R2"], kernel_param["P1"], kernel_param["P2"]):
            kernel_matrix = make_kernel_matrix(
                data,
                model=m,
                kernel_dosage=kernel_for_dosage,
                kernel_abSignals=kernel_for_abSignal,
                scale=scale,
            )
            output_file_name = "kernel_matrix_SelectiveSet_SRP" + "_SA_" + str(m[0]) + "_SO_" + str(m[1]) +\
                               "_R1_" + str('{:.1E}'.format(m[2])) + "_R2_" + str(m[3]) +\
                               "_P1_" + str(m[4]) + "_P2_" + str(m[5]) + ".npy"
            np.save(output_file_name, kernel_matrix)
            
    elif kernel_for_dosage == "poly_kernel" and kernel_for_abSignal == "poly_kernel":
         for m in product(kernel_param["SA"], kernel_param["SO"], kernel_param["R1"], kernel_param["R2"], kernel_param["P1"], kernel_param["P2"]):
            kernel_matrix = make_kernel_matrix(
                data,
                model=m,
                kernel_dosage=kernel_for_dosage,
                kernel_abSignals=kernel_for_abSignal,
                scale=scale,
            )
            output_file_name = "kernel_matrix_SelectiveSet_SPP" + "_SA_" + str(m[0]) + "_SO_" + str(m[1]) +\
                               "_R1_" + str(m[2]) + "_R2_" + str(m[3]) +\
                               "_P1_" + str(m[4]) + "_P2_" + str(m[5]) + ".npy"
            np.save(output_file_name, kernel_matrix) 
    

In [37]:
data_malaria_sorted = sort(data_malaria)

In [38]:
data_malaria_sorted

,Patient,group,Protection,Dose,TimePointOrder,mal_mito_3.iso1.exon1.amp1_4066,mal_mito_2.iso1.exon1.amp1_1466,mal_mito_1.iso1.exon1.amp1_3777,PFL2430c_484,PFL2215w_4059,...,MAL8P1.320_1o1_4243,MAL8P1.156_187,MAL8P1.155_2o3_4890,MAL8P1.143_3o15_2092,MAL8P1.143_1o15_2416,MAL8P1.143_14o15_1450,MAL8P1.140_4049,MAL7P1.32_5037,MAL13P1.303e4s1_1829,MAL13P1.303e1s1_4421
0,T2-002 III 14,2,1,1,2,7.033423,6.357552,6.954196,3.700440,0.000000,...,0.000000,0.000000,0.000000,11.168672,9.628446,0.000000,0.000000,15.327447,0.000000,0.000000
1,T2-005 III 14,5,0,0,2,8.596190,8.481799,8.840778,9.076816,9.045760,...,8.663558,0.000000,9.177420,9.353147,7.632995,0.000000,8.290019,8.556506,8.873444,8.961450
2,T2-006 III 14,6,1,1,2,9.838416,9.589651,10.115694,10.676398,11.304922,...,10.879966,10.665336,10.838416,10.771489,10.681678,10.901621,10.989394,13.195757,11.017504,10.706064
3,T2-007 III 14,7,0,1,2,6.965784,7.451211,7.149747,10.499846,5.066089,...,0.000000,7.396605,0.000000,11.946724,8.339850,8.497852,0.000000,15.761292,6.209453,0.000000
4,T2-010 III 14,10,0,2,2,6.483816,7.977280,7.800900,10.444497,9.814582,...,8.147205,10.837628,10.098032,11.304351,10.863799,10.425740,10.063395,14.122383,10.745674,0.000000
5,T2-011 III 14,11,0,0,2,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,T2-012 III 14,12,0,1,2,7.027906,7.499846,7.974415,9.063395,0.000000,...,7.388017,8.445015,9.463524,10.251482,0.000000,9.923327,0.000000,12.406736,8.828136,8.560333
7,T2-015 III 14,15,1,1,2,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,7.409391,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,T2-017 III 14,17,1,3,2,0.000000,0.000000,0.000000,0.000000,3.584963,...,7.751544,6.820179,8.197217,6.651052,4.459432,7.422065,7.434628,7.312883,5.870365,7.266787
9,T2-018 III 14,18,1,2,2,0.000000,0.000000,0.000000,0.000000,0.000000,...,9.310613,0.000000,0.000000,0.000000,0.000000,0.000000,8.618386,0.000000,0.000000,0.000000


In [170]:
main(data_malaria_sorted, kernel_comb_rbf_rbf, "rbf_kernel", "rbf_kernel")

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
80
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
80
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
118
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
86
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
81
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
80
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
80
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
80
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
80
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
80
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
80
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
80
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kerne

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

...

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

In [171]:
main(data_malaria_sorted, kernel_comb_rbf_poly, kernel_for_dosage="rbf_kernel", kernel_for_abSignal="poly_kernel")

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

In [172]:
main(data_malaria_sorted, kernel_comb_poly_rbf, kernel_for_dosage="poly_kernel", kernel_for_abSignal="rbf_kernel")

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

In [173]:
main(data_malaria_sorted, kernel_comb_poly_poly, kernel_for_dosage="poly_kernel", kernel_for_abSignal="poly_kernel")

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

In [52]:
target_label_vec = data_malaria_sorted.loc[:, "Patient":"TimePointOrder"]

In [53]:
target_label_vec

,Patient,group,Protection,Dose,TimePointOrder
0,T2-002 III 14,2,1,1,2
1,T2-005 III 14,5,0,0,2
2,T2-006 III 14,6,1,1,2
3,T2-007 III 14,7,0,1,2
4,T2-010 III 14,10,0,2,2
5,T2-011 III 14,11,0,0,2
6,T2-012 III 14,12,0,1,2
7,T2-015 III 14,15,1,1,2
8,T2-017 III 14,17,1,3,2
9,T2-018 III 14,18,1,2,2


In [54]:
target_label_vec.to_csv("target_label_vec.csv")

In [57]:
selected_data_malaria_sorted = sort(selected_data_malaria)

In [58]:
selected_data_malaria_sorted

,Patient,group,Protection,Dose,TimePointOrder,PFE1590w_2119,PFD0425w_2780,PFC0210c_2763,PFB0300c_4064,PF3D7_1480000.iso1.exon2.amp1_2445,...,PF3D7_0100100.iso1.exon1.amp2_1950,PF3D7_0100100.iso1.exon1.amp1_1661,PF14_0540_1132,PF13_0201_3400,PF13_0197_176,PF13_0172_4696,PF13_0012_499,PF11_0344_1786,PF10_0088_4048,PF08_0013_1803
0,T2-002 III 14,2,1,1,2,0.000000,0.000000,8.763212,12.830218,8.477758,...,0.000000,2.906891,15.156320,10.179909,11.091435,0.000000,0.000000,0.000000,0.000000,0.000000
1,T2-005 III 14,5,0,0,2,7.622052,9.084808,0.000000,7.994353,9.958553,...,8.980140,8.544964,0.000000,9.025140,7.724514,8.479780,9.120238,7.741467,8.396605,7.703904
2,T2-006 III 14,6,1,1,2,10.630722,11.119590,11.304922,10.851749,9.967226,...,9.948367,10.015415,12.196448,11.115694,10.629812,10.513728,10.396070,10.622967,11.139232,10.958190
3,T2-007 III 14,7,0,1,2,10.572227,0.000000,6.839204,12.678050,6.375039,...,7.413628,6.179909,14.066762,8.019591,7.665336,6.257388,6.562242,8.044394,0.000000,7.335390
4,T2-010 III 14,10,0,2,2,9.475733,5.870365,10.083479,12.193371,8.596190,...,7.873444,7.204571,13.779052,10.607793,10.998590,10.735556,10.178665,10.163650,8.736402,10.590587
5,T2-011 III 14,11,0,0,2,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,T2-012 III 14,12,0,1,2,9.451211,6.507795,8.566054,0.000000,8.638436,...,7.471675,7.290019,12.040290,5.727920,8.815383,8.095397,8.400879,9.098032,7.727920,9.183635
7,T2-015 III 14,15,1,1,2,0.000000,0.000000,9.214319,12.052568,0.000000,...,0.000000,0.000000,0.000000,0.000000,8.216746,0.000000,0.000000,8.475733,0.000000,0.000000
8,T2-017 III 14,17,1,3,2,0.000000,8.063395,13.522398,0.000000,0.000000,...,0.000000,0.000000,7.794416,6.409391,6.066089,5.954196,6.228819,7.335390,7.714246,6.748193
9,T2-018 III 14,18,1,2,2,0.000000,7.960002,10.206404,10.287712,0.000000,...,6.348728,4.882643,0.000000,0.000000,0.000000,7.611025,0.000000,0.000000,7.643856,0.000000


In [174]:
main_selectiveSet(data=selected_data_malaria_sorted, kernel_param=kernel_comb_rbf_rbf, kernel_for_abSignal="rbf_kernel", kernel_for_dosage="rbf_kernel")

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
106
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
82
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
80
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
80
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
80
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
80
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
80
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
80
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
80
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of ke

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

...

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

In [175]:
main_selectiveSet(data=selected_data_malaria_sorted, kernel_param=kernel_comb_rbf_poly, kernel_for_abSignal="poly_kernel", kernel_for_dosage="rbf_kernel")

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120

In [176]:
main_selectiveSet(data=selected_data_malaria_sorted, kernel_param=kernel_comb_poly_rbf, kernel_for_abSignal="rbf_kernel", kernel_for_dosage="poly_kernel")

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

In [177]:
main_selectiveSet(data=selected_data_malaria_sorted, kernel_param=kernel_comb_poly_poly, kernel_for_abSignal="poly_kernel", kernel_for_dosage="poly_kernel")

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra

Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
119
Dimension of kernel matrix
(120, 120)
Rank of kernel matrix
120
Dimension of kernel matrix
(120, 120)
Ra